In [14]:
import sys
import os
import numpy as np
import shutil

In [15]:
output_root = '/scratch/op13/dedalus/MRBC/' 
exec_file = '/home/op13/python/dedalus/MRBC/MRBC_3D.py'

In [16]:
param_name = 'Q0'
#param_values = ['0.05','0.01','0.015','0.02','0.025','0.03','0.035','0.04','0.045','0.05','0.055','0.06','0.08','0.1']
param_values = ['0.0','0.01','0.02']
#param_values = ['0.015']

In [20]:
namelist = \
    "Lx, Ly, Lz = 8, 8, 1\n"+\
    "Nx, Ny, Nz = 256, 256, 32\n"+\
    "MRayleigh = 1e6\n"+\
    "M0 = 0\n"+\
    "D0 = 0\n"+\
    "MH = -1.0\n"+\
    "DH = 0.5\n"+\
    "N2 = 1.5\n"+\
    "Q0 = 0.00\n"+\
    "free_slip_upper = True\n"+\
    "Chi = 2.0\n"+\
    "compute_mesh = (16,16)\n"+\
    "stop_sim_time = 200.0\n"
exec(namelist)
print(namelist)

Lx, Ly, Lz = 8, 8, 1
Nx, Ny, Nz = 256, 256, 32
MRayleigh = 1e6
M0 = 0
D0 = 0
MH = -1.0
DH = 0.5
N2 = 1.5
Q0 = 0.00
free_slip_upper = True
Chi = 2.0
compute_mesh = (16,16)
stop_sim_time = 200.0



In [21]:
script_start = [  \
    "#!/bin/bash\n",\
    "#SBATCH --nodes=1\n",\
    "#SBATCH --tasks-per-node=16\n",\
    "#SBATCH --cpus-per-task=16\n",\
    "#SBATCH --mem=64GB\n",\
    "#SBATCH --time=06:00:00\n",\
    "#SBATCH --job-name=MRBC_baro\n",\
    "#SBATCH --mail-type=END\n",\
    "#SBATCH --mail-user=op13@nyu.edu\n",\
    "#SBATCH --output=MRBC_2D_slurm_%j.out\n",
    "module purge\n"]

script_end = [ \
              "srun /scratch/work/public/singularity/run-dedalus-3.0.0a0.bash python MRBC.py"]


In [22]:
file_launch = output_root + 'launch_slurm.sh'
launcher = open(file_launch,'w')
for value in param_values:
    exec(param_name + '=' + value)
    expname = 'MRBC_3D_MRa_'+"{:.1e}".format(MRayleigh) +'_N2S_'+"{:.1e}".format(N2) +'_Q0_'+"{:.1e}".format(Q0) \
            +  '_Lx_'+"{:.1e}".format(Lx) 
    exp_dir = output_root + expname
    if (not os.path.exists(exp_dir)):
        os.makedirs(exp_dir, exist_ok = True)
    os.chdir(exp_dir)
    shutil.copyfile(exec_file,exp_dir+'/MRBC.py')
    nml_filename = 'MRBC3D_param.py'
    nml_file = open(nml_filename, 'w')
    nml_file.writelines(namelist)
    nml_file.writelines(param_name + '=' + value +'\n')
    nml_file.close()
    script_name = 'run_MRBC.slurm'
    script = open(script_name,'w')
    script.writelines(script_start)
    script.writelines('cd '+exp_dir+'\n')
    script.writelines(script_end)
    script.close()
    launcher.writelines("sbatch ./" + expname + "/" + script_name +'\n')
launcher.close()

print('Jobs can be submitted to the queue by going to the directory:\n' + output_root)
print('and enter the command: \nsource launch_slurm.sh')

Jobs can be submitted to the queue by going to the directory:
/scratch/op13/dedalus/MRBC/
and enter the command: 
source launch_slurm.sh
